# Metodologia
GANS e Redes Siamesas se dão muito bem para discriminar uma imagem de outra optei por redes siamesas devido a um custo menor de processamento se equiparado as GANS.
Criar um Mapa de Features extraídos da imagem.
Utilizar Distância Euclidiana entre um mapa de features e outro.
<h5>Arquitetura do Modelo</h5>
Google Colab Notebook em Anexo
<h5>Pré-Processamento</h5>
Devido a alta resolução da imagem optei por reduzir para 224x224 mantendo o aspect ratio e preenchendo as imagens de resoluções variáveis com (Zero Padding)
<h5>Observações:</h5>
Devido a limitação de dados de treino não pude pensar em outra metodolgia que gere bons resultados (fiz algumas pesquisas sobre modelos de ML para assinaturas mas não ví nada satisfatório)

In [122]:
import cv2
import numpy as np
import imutils
import random
from itertools import cycle
import glob

In [66]:
img = cv2.imread("TrainingSet/Reference/R001.png", 0)

In [67]:
img_resized = imutils.resize(img, 224, 224)

In [143]:
img_resized.shape

(159, 224)

In [192]:
cv2.imshow("teste",zero_padding(img_resized, 224, 224))
cv2.waitKey(0)
cv2.destroyAllWindows()
print(zero_padding(img_resized, 224, 224).shape)

(224, 224)


In [161]:
def zero_padding(image, width, height):
    new_size = image.shape[:2]
    delta_w = width - new_size[1]
    delta_h = height - new_size[0]
    if delta_h ==0 and delta_w ==0:
        return image
    else:
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)
        color = (0, 0, 0)
        new_im = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)
        return new_im

In [75]:
def add_noise(image,prob):
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output

In [1]:
def load_data(files, width, height, debug=False, augmented=False, percent=30):
    X = []
    percent = int(len(files)*(percent/100))
    noise_range =cycle(list(range(1, 5)))
    for file in files:
        im = cv2.imread(file)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        im = imutils.resize(im, width, height)
        im = zero_padding(im, width, height)
        if augmented is True:
            noise_rate = next(noise_range)/10
            noise_img = sp_noise(im, noise_rate)
            noise_img = zero_padding(noise_img, width, height)
        if debug is True and augmented is False:
            cv2.imshow("Original",im)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            debug = False
        if augmented is True and debug is True:
            cv2.imshow("Original A",im)
            cv2.imshow("Augmented",noise_img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            print(file)
            print(im.shape)
            print(noise_img.shape)
            debug = False
        im - im.astype(np.float32)
        im = im / 255.0
        X.append(im)
        noise_img = noise_img / 255.0
        X.append(noise_img)
        percent-=1
        if percent == 0:
            augmented = False
    X = np.array(X)
    print(X.shape)
    train_x = X.reshape(X.shape[0], width, height, 1)
    return train_x

In [200]:
files = glob.glob("TrainingSet/Reference/*.png")
len(files)

9

In [201]:
data = load_data(files, 224, 224, True, True, 25)

TrainingSet/Reference\R001.png
(224, 224)
(224, 224)
(18, 224, 224)
